# 환경설정
- 구글 코랩

In [1]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [2]:
strategy = tf.distribute.TPUStrategy(resolver)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Hanium_private/model/

/content/drive/MyDrive/Hanium_private/model


# KoGPT2

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00


In [6]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2Model
import sqlite3
import numpy as np
import heapq

## 모델 및 토크나이저 초기화

In [7]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [8]:
# 모델 및 토크나이저 초기화
MODEL_NAME = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME,
                                          bos_token=BOS, eos_token=EOS, unk_token='<unk>',
                                          pad_token=PAD, mask_token=MASK)
model = GPT2Model.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
model.eval()

GPT2Model(
  (wte): Embedding(51200, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

## 데이터 로드

In [11]:
# 데이터 로드
embeddings = torch.load('data/embeddings.pt')
passage_ids = torch.load('data/passage_ids.pt')

### 예시

In [16]:
len(embeddings)

2000

In [18]:
len(passage_ids)

2000

## embedding vector 기반 유사도 측정
- 데이터셋의 ebedding vector 로드
- 입력 embedding vector와 유사도 측정

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
def create_embedding(text):
    input_ids = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding='max_length').to(device)

    with torch.no_grad():
        output = model(**input_ids)
    last_hidden_state=output.last_hidden_state

    # mean pooling
    attention_mask = input_ids.attention_mask
    num_of_tokens = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

    sum_embeddings = torch.sum(last_hidden_state * num_of_tokens, 1)

    total_num_of_tokens = num_of_tokens.sum(1)
    total_num_of_tokens = torch.clamp(total_num_of_tokens, min=1e-9)

    result = sum_embeddings / total_num_of_tokens
    return result.squeeze()

In [162]:
def recommend_books(text, k=5):
    # 입력 문장의 임베딩 계산
    input_embedding = create_embedding(text)
    book_heap = []
    for idx, embed in enumerate(embeddings):
        similarity = torch.nn.functional.cosine_similarity(input_embedding.unsqueeze(0), embed.unsqueeze(0))
        if len(book_heap) < k:
            heapq.heappush(book_heap, (similarity.item(), passage_ids[idx]))
        else:
            if similarity > book_heap[0][0]:
                heapq.heappop(book_heap)
                heapq.heappush(book_heap, (similarity.item(), passage_ids[idx]))
    # 가장 유사한 도서들을 유사도 내림차순으로 정렬
    book_heap.sort(key=lambda x: (-x[0], x[1]))

    return book_heap # [passage_id for _, passage_id in book_heap] # book_heap: [(유사도, id), (유사도, id)]

In [163]:
def get_book_title_by_id(passage_id):
    """ 주어진 passage_id에 해당하는 도서의 제목을 반환"""
    with sqlite3.connect('data/books.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT title FROM sample WHERE id=?", (passage_id,))
        title = cursor.fetchone()[0]
    return title

In [215]:
def get_db_by_id(passage_id, feature="title"):
    info = None
    """ 주어진 passage_id에 해당하는 도서의 제목을 반환"""
    with sqlite3.connect('data/books.db') as conn:
        cursor = conn.cursor()
        cursor.execute(f"SELECT {feature} FROM sample WHERE id=?", (passage_id,))
        if feature=="title":
            info = cursor.fetchone()[0]
        if feature=="*":
            info = cursor.fetchone()[2:]
            info = '\n >> '.join(info)
    return info

# Test
- GPT 기반 맥락 이해형 도서 추천 및 검색을 테스트

In [168]:
input_text = input("문장을 입력해주세요: ") # 일본 애니메이션을 만드는 사람들의 저작권을 어떻게 받을까? / 일본 애니메이션을 만드는 사람들의 저작권은 어떤 권리인가?
k = int(input("추천 도서 개수를 입력하세요 (k): "))
k_book = recommend_books(input_text, k)

문장을 입력해주세요: 일본 애니메이션을 만드는 사람들의 저작권은 어떤 권리인가?
추천 도서 개수를 입력하세요 (k): 5


In [170]:
print(f"{k}개의 추천 도서:")
for i, (similarity, id) in enumerate(k_book, start=1):
    print(f"{i}. {get_book_title_by_id(id)} (유사도: {similarity:.2f})")

5개의 추천 도서:
1. 콘텐츠 제작 네트워크 구축 지원방안 연구 (유사도: 0.73)
2. 애니메이션 지원정책의 효율성 제고 방안 연구 (유사도: 0.71)
3. 공존, 지구촌을 살리는 위대한 나눔, 적정기술 (유사도: 0.70)
4. 일본 애니메이션 산업 현황과 한 · 일 공동제작 모델 연구 (유사도: 0.70)
5. 한국영화수익성분석 (유사도: 0.69)


In [216]:
feature = "*"
for i, (similarity, id) in enumerate(k_book, start=1):
    print(f"{i}. {get_db_by_id(id)} (유사도: {similarity:.2f})")
    print(f"{get_db_by_id(id, feature)}")

1. 콘텐츠 제작 네트워크 구축 지원방안 연구 (유사도: 0.73)
콘텐츠 제작 네트워크 구축 지원방안 연구
 >> 일본 동영상협회의 자료에 따르면, 일본 애니메이션 제작 회사 전체의 매출은 2015년 1365억 엔에 비해 2016년에는 2088억 엔, 2017년에는 2244억 엔으로 증가하였다. 높은 성장률을 이룩한 애니메이션 산업이 일본 경제의 새로운 성장 분야로 주목받은 이유는 지적재산권 관련 산업, 콘텐츠 산업에 대한 관심이 높아졌기 때문이다. 애니메이션은 영상 자체 사업뿐만 아니라, 작품과 캐릭터를 이용한 2차 사업 확대 가능성이 크기 때문에 성장 산업으로 기대를 받았다. 그 결과는 애니메이션 제작에 투입되는 자금의 증가와 제작의 확대로 이어졌고 타 업종으로부터의 신규 참가가 많았던 점도 특징이다. 또한 일본의 콘텐츠 산업은 콘텐츠보다 미디어 측이 주도권을 쥐고 있는데 그러한 미디어 산업의 중심에 있는 것이 방송국으로 TV 방송국, 신문사, 출판사 등이 제작 위원회이며 저작권 소유자이다.
 >> 박찬욱
 >> 문화체육관광일반
 >> 또한 일본의 콘텐츠 산업은 콘텐츠보다 미디어 측이 주도권을 쥐고 있는데 그러한 미디어 산업의 중심에 있는 것이 방송국으로 TV 방송국, 신문사, 출판사 등이 제작 위원회이며 저작권 소유자이다.
2. 애니메이션 지원정책의 효율성 제고 방안 연구 (유사도: 0.71)
애니메이션 지원정책의 효율성 제고 방안 연구
 >> 애니메이션의 수익성에 대한 불안감, 그리고 애니메이션 장르에 대한 사회적으로 미성숙한 편견 등의 이유로 애니메이션은 실사영화보다 배급망을 확보하기 더욱 어려운 조건이다. 일례로 이현세 원작을 애니메이션화한 <아마게돈>의 경우 극장 상영을 위해 서울극장 측에 개런티를 지불해야 했는데 1천만 원에서 2천만 원에 달하는 이 개런티는 시장에 본격적으로 진출한 경험이 없는 애니메이션에 대한 일종의 보험료 성격이 강하다. 애니메이션계의 많은 관심 속에 제작된 <아마게돈>이 단지 애니메이션이라는 이유로 실사영화와 차별을 받는